In [1]:
# script to write xcp-compatible task event timing arrays convolved with HRF 

#writing for one run-01_events.tsv, use for loop to iterate across all events.tsvs in future
# & potentially for each subj (depending on whether xcp can recognize BIDS inheritence)

import pandas as pd
import numpy as np

In [2]:
#ONSET TIMES IN STICK FILES REFLECT FACT THAT ANALYSIS PIPELINE DELETED FIRST 20 SECONDS=10TR OF BOLD RUNS, 
#DURING WHICH TWO "DUMMY" TASK TRIALS OCCURRED

sandbox='/cbica/projects/wolf_satterthwaite_reward/Margaret/Day2/curation/code/sandbox/'

dfr1=pd.read_csv('/cbica/projects/wolf_satterthwaite_reward/Margaret/Day2/curation/BIDS/task-card_run-01_events.tsv', sep ="\t")
dfr1.head(5)

,onset,duration,trial_type
0,0,2,cue
1,2,4,anticipation
2,6,2,win_outcome
3,12,2,cue
4,14,2,anticipation


In [3]:
dfr1.duration.unique()

array([ 2,  4, 10, 12,  8,  6])

In [4]:
#iterate to add new rows accounting for conditions lasting more than one TR (duration>2 sec)
for index in dfr1.index:
    if dfr1.loc[index, 'duration'] > 2.0:
        x=int(dfr1.loc[index, 'duration'])
        print(x)

4
10
12
4
8
8
10
12
4
6
6
6
6
4


In [5]:
for index in dfr1.index:
    if dfr1.loc[index, 'duration'] > 2.0:
        y=int(dfr1.loc[index, 'duration'])
        z=int(y/2)
        for x in range(1, z):
            on=int(dfr1.loc[index,'onset'])+(2.0*x)
            t=dfr1.loc[index,'trial_type']
            dfr1=dfr1.append({'onset': on,'trial_type':t}, ignore_index=True)

In [6]:
dfr1=dfr1.sort_values(by=['onset'])
dfr1

,onset,duration,trial_type
0,0.0,2.0,cue
1,2.0,4.0,anticipation
72,4.0,NaN,anticipation
2,6.0,2.0,win_outcome
3,12.0,2.0,cue
...,...,...,...
107,306.0,NaN,anticipation
68,308.0,2.0,lose_outcome
69,318.0,2.0,cue
70,320.0,2.0,anticipation


In [7]:
#writing to confirm output as expected
with open(sandbox+'dfr1.csv', 'w') as f:
    f.write(dfr1.to_csv(index=False))

In [8]:
#return num rows needed for task array from stick files
## to do: replace 2 with func to define TR directly from jsons
num_trs=int((dfr1["onset"].max())/2)
print(num_trs)

161


In [9]:
#write list of all TRs
trs=[]
for x in range(0, int(dfr1["onset"].max()+2) ,2):
    trs.append(x)
    
#convert to df
task=pd.DataFrame(trs, columns=['tr'])

In [10]:
#iterate through conditions and merge into list of TRs
for condit in dfr1.trial_type.unique():
    df=dfr1.loc[dfr1['trial_type']==condit]
    df=df.rename(columns={'duration':'dur_'+condit, 'trial_type':condit})
    task=task.merge(df, how='left', left_on='tr', right_on='onset')
    task=task.drop(columns=['onset'])

In [11]:
task.head(10)

,tr,dur_cue,cue,dur_anticipation,anticipation,dur_win_outcome,win_outcome,dur_lose_outcome,lose_outcome
0,0,2.0,cue,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,NaN,4.0,anticipation,NaN,NaN,NaN,NaN
2,4,NaN,NaN,NaN,anticipation,NaN,NaN,NaN,NaN
3,6,NaN,NaN,NaN,NaN,2.0,win_outcome,NaN,NaN
4,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,12,2.0,cue,NaN,NaN,NaN,NaN,NaN,NaN
7,14,NaN,NaN,2.0,anticipation,NaN,NaN,NaN,NaN
8,16,NaN,NaN,NaN,NaN,2.0,win_outcome,NaN,NaN
9,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
#writing to confirm output as expected
with open(sandbox+'tasks.csv', 'w') as f:
    f.write(task.to_csv(index=False))